# Acquiring and Cleaning Metadata


Last Updated 31st July 2023

Note: As of 31st July 2023, slight changes in NCBI's command line tools have impacted the way these scripts run. The file ```metadata-sample.csv``` was acquired with a prior version of NCBI's dataformat and can be directly used to run the ```Boxplot``` and ```PlottingContigN50``` scripts. This following section provides updated details on how to create a similar file. If there are any unexpected difficulties in using NCBI's command line tools, check to make sure that both ```datasets``` and ```dataformat``` is up to date. 

## 1. Using NCBI's Command Line Tools 
NCBI has published command line tools to search and pull metadata. Running this code will result in a .tsv file, which can then adapted using the following script to prepare for visualisation in Python. 

The metadata is pulled using the ```datasets``` tool with the following line: 

```datasets summary genome taxon spiralia --as-json-lines``` 

This is then piped into the ```dataformat``` tool which reformats the output to make it easier to use. 
```dataformat tsv genome``` 

Without any flags, NCBI provides all fields by default. Certain tags can be chosen under the ```--fields``` flag to request specific columns so that the resulting table is more manageable. 

The final command including fields is shown below. The result is saved into a .tsv file to be fed into pandas.  

>```datasets summary genome taxon spiralia --as-json-lines | dataformat tsv genome --fields accession,organism-name,organism-common-name,organism-tax-id,assminfo-submitter,assminfo-release-date,assminfo-sequencing-tech,assminfo-level,assmstats-contig-l50,assmstats-contig-n50,assmstats-gc-count,assmstats-gc-percent,assmstats-number-of-component-sequences,assmstats-number-of-contigs,assmstats-number-of-scaffolds,assmstats-scaffold-l50,assmstats-scaffold-n50,assmstats-total-number-of-chromosomes,assmstats-total-sequence-len,assmstats-total-ungapped-len,organism-infraspecific-sex,source_database,wgs-project-accession > ncbi_output.tsv```


## 2. Cleaning up 

#### Importing packages: 
We use ```numpy``` and ```pandas``` for managing our data, ```requests```, ```concurrent.futures```, and ```sys``` to crawl extra metadata from the NCBI website, and ```Bio Entrez``` to convert Organism Tax ID to phyla. 

In [1]:
import numpy as np
import pandas as pd
import requests 
import concurrent.futures 
# from concurrent.futures import Future 
import sys
from Bio import Entrez 

#### Helper functions: 
I wrote a series of additional helper functions throughout the process to streamline the code. These include a class Metadata to crawl additional information from NCBI as well as some functions for eliminating duplicates. Running the file does not execute anything, but allows for the functions to be used further down the line. 

In [2]:
%run helper_functions.ipynb

In [3]:
in_data = processNcbiMetadata('ncbi_output.tsv') 
out_file = 'metadata.csv'  

#### Metadata Class: 
Taking the tsv output of NCBI's command line tools, we use the Metadata class to find additional information. In particular, we're interested in looking at the earliest assembly for each dataset. For any accession ending in a number other than 1, we use ```firstPub()``` to pull the date of publication and contig n50 for the earliest assembly available on NCBI (same accession but ending with .1 instead). The metadata also has functions ```threadCreep``` and ```taxCreep``` to pull additional information from NCBI. ```taxCreep``` acts as a slower alternative to the process written below, but has been more reliable in certain cases. 

In [4]:
crawl_metadata = Metadata(in_data) 

In [5]:
#crawl_metadata.threadCreep()
#crawl_metadata.taxCreep()

In [6]:
# Note that in newer versions of NCBI's command line tools, the submission date
#   column has been replaced by more precise date columns. 
#   In this update, the firstPub function will reference 'Release_Date' by default
#   but this can be changed to 'Submission_Date' parameter if using older datasets 
crawl_metadata.firstPub(date_column = 'Release_Date')

In [7]:
metadata = crawl_metadata.table

#### Finding Phyla: 
We use Biopython to get the corresponding phylum for each entry based on Tax ID. 

In [8]:
Entrez.email = '<insert email here>'

In [9]:
taxids = metadata.Organism_Taxonomic_ID
handle = Entrez.efetch(db = 'taxonomy', id = taxids, retmode = 'xml') 
records = Entrez.read(handle) 


In [10]:
phyla = [entry['ScientificName'] for record in records 
         for entry in record['LineageEx'] if 'phylum' in entry.values()]

In [11]:
metadata['Phylum'] = phyla 

#### Long reads or Short reads? 
Some inconsistencies in captialisation and detail in the 'Sequencing_Tech' column make it difficult to work with. As such, we add an additional column called 'Sequencing_Type' with three distinct categories: long reads, short reads, and no information provided. 

In [12]:
metadata = readType(metadata) 

#### Looking at our Dataframe: 

In [13]:
# Set Pandas such that all columns on a dataframe can be seen without truncation. 
pd.set_option('display.max_columns', None) 

In [14]:
metadata.loc[metadata.Phylum == 'Phoronida'] 

,Organism_Name,Organism_Common_Name,Organism_Taxonomic_ID,Submitter,Release_Date,Sequencing_Tech,Level,Contig_L50,Contig_N50,GC_Count,GC_Percent,Number_of_Component_Sequences,Number_of_Contigs,Number_of_Scaffolds,Scaffold_L50,Scaffold_N50,Total_Number_of_Chromosomes,Total_Sequence_Length,Total_Ungapped_Length,Organism_Infraspecific_Names_Sex,Source_Database,WGS_project_accession,First_Publication_Date,Original_Contig_N50,Phylum,Sequencing_Type
Accession,,,,,,,,,,,,,,,,,,,,,,,,,,
GCA_002633005.1,Phoronis australis,NaN,115415,Okinawa Institute of Science and Technology Gr...,2017-10-23,454; Illumina MiSeq; Illumina HiSeq,Scaffold,2133,68151,192012380,39.0,3983,15468,3983.0,226.0,655058.0,NaN,498443662,488114166,NaN,SOURCE_DATABASE_GENBANK,NMRA01,2017-10-23,68151,Phoronida,Short read
GCA_028565635.1,Phoronis ovalis,NaN,492055,Iridian Genomes,2023-02-09,Illumina,Scaffold,17834,4788,110948516,34.0,113601,123365,113601.0,14961.0,5749.0,NaN,326042390,325077420,NaN,SOURCE_DATABASE_GENBANK,JAOXYB01,2023-02-09,4788,Phoronida,Short read
GCA_028565715.1,Phoronis pallida,NaN,492056,Iridian Genomes,2023-02-09,Illumina,Scaffold,52451,2318,153668160,36.0,257561,260149,257561.0,51547.0,2362.0,NaN,425364260,425146860,NaN,SOURCE_DATABASE_GENBANK,JAOXYC01,2023-02-09,2318,Phoronida,Short read


In [15]:
metadata.loc[metadata.Organism_Name == 'Lingula anatina'] 

,Organism_Name,Organism_Common_Name,Organism_Taxonomic_ID,Submitter,Release_Date,Sequencing_Tech,Level,Contig_L50,Contig_N50,GC_Count,GC_Percent,Number_of_Component_Sequences,Number_of_Contigs,Number_of_Scaffolds,Scaffold_L50,Scaffold_N50,Total_Number_of_Chromosomes,Total_Sequence_Length,Total_Ungapped_Length,Organism_Infraspecific_Names_Sex,Source_Database,WGS_project_accession,First_Publication_Date,Original_Contig_N50,Phylum,Sequencing_Type
Accession,,,,,,,,,,,,,,,,,,,,,,,,,,
GCF_001039355.2,Lingula anatina,NaN,7574,Okinawa Institute of Science and Technology Gr...,2018-01-26,454; Illumina MiSeq; Illumina HiSeq; PacBio,Scaffold,1942,56068,141655982,36.0,2677,13877,2677.0,271.0,460090.0,NaN,406282338,388995278,male,SOURCE_DATABASE_REFSEQ,LFEI02,2018-01-26,56068,Brachiopoda,Long read
GCA_001039355.2,Lingula anatina,NaN,7574,Okinawa Institute of Science and Technology Gr...,2018-01-26,454; Illumina MiSeq; Illumina HiSeq; PacBio,Scaffold,1942,56068,141655982,36.0,2677,13877,2677.0,271.0,460090.0,NaN,406282338,388995278,male,SOURCE_DATABASE_GENBANK,LFEI02,2018-01-26,56068,Brachiopoda,Long read


#### Manually Fixing Enries: 
Some of the earlier genome assemblies may be missing information on NCBI. We try to fix these manually to paint the most accurate picture possible. 

In [16]:
metadata.loc[metadata.Organism_Name=="Schistosoma mansoni", "First_Publication_Date"] = "2009-07-16"
metadata.loc[metadata.Organism_Name=="Schistosoma mansoni", "Original_Contig_N50"] = "77000" 

#### Saving our Dataframe: 
We save the dataframe to a csv to use in the following visualisation stages. 

In [17]:
metadata.to_csv(out_file)